<a href="https://colab.research.google.com/github/PIKAPIKAAAA/python-for-trading-master/blob/main/portfolio_optimization_2_assets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 7.1 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [8]:
import yfinance as yf
import datetime as dt
import pandas as pd
import numpy as np

In [9]:
symbols = ['GOOG', 'CAT']
df = yf.download(symbols, start='2005-01-01', end='2015-01-01', interval='1mo')['Adj Close'].dropna()

[*********************100%***********************]  2 of 2 completed


In [10]:
df

,CAT,GOOG
Date,,
2005-01-01,28.513189,97.444756
2005-02-01,30.550947,93.643997
2005-03-01,29.390635,89.917969
2005-04-01,28.301003,109.589233
2005-05-01,30.390352,138.117310
...,...,...
2014-08-01,87.975266,570.034973
2014-09-01,79.877045,575.779175
2014-10-01,81.796776,557.549255


# **Add_return_columns**

In [11]:
start_price = 100
end_price = 120
profit = end_price / start_price - 1
profit

0.19999999999999996

In [12]:
def add_retun_columns(symbols):
  #cxrilshi cifrebis shemokleba/damrgvaleba
  pd.options.display.float_format = '{:.3f}'.format
  for symbol in symbols:
    #start price / end price - 1
    df[f'{symbol}_Ret'] = df[symbol] / df[symbol].shift(1) - 1


In [13]:
add_retun_columns(symbols)

In [14]:
df.dropna()

,CAT,GOOG,GOOG_Ret,CAT_Ret
Date,,,,
2005-02-01,30.551,93.644,-0.039,0.071
2005-03-01,29.391,89.918,-0.040,-0.038
2005-04-01,28.301,109.589,0.219,-0.037
2005-05-01,30.390,138.117,0.260,0.074
2005-06-01,30.778,146.526,0.061,0.013
...,...,...,...,...
2014-08-01,87.975,570.035,0.000,0.089
2014-09-01,79.877,575.779,0.010,-0.092
2014-10-01,81.797,557.549,-0.032,0.024


In [15]:
df = df[['GOOG_Ret', 'CAT_Ret']].dropna()
df.corr()

,GOOG_Ret,CAT_Ret
GOOG_Ret,1.000,0.196
CAT_Ret,0.196,1.000


# **Standard_Deviation**

In [16]:
prices = df['CAT_Ret'][:5]

In [17]:
prices

Date
2005-02-01    0.071
2005-03-01   -0.038
2005-04-01   -0.037
2005-05-01    0.074
2005-06-01    0.013
Name: CAT_Ret, dtype: float64

In [18]:
prices.std()

0.055135931370190766

STD: step 1

In [19]:
prices_sum = sum(prices)

In [20]:
prices_sum

0.08299044983701143

In [21]:
mean = prices_sum / 5

In [22]:
mean

0.016598089967402287

Std: step 2

In [23]:
deviation_from_the_mean = prices - mean 

In [24]:
deviation_from_the_mean

Date
2005-02-01    0.055
2005-03-01   -0.055
2005-04-01   -0.054
2005-05-01    0.057
2005-06-01   -0.004
Name: CAT_Ret, dtype: float64

STD: step 3

In [25]:
deviation_squares = deviation_from_the_mean ** 2

In [26]:
deviation_squares

Date
2005-02-01   0.003
2005-03-01   0.003
2005-04-01   0.003
2005-05-01   0.003
2005-06-01   0.000
Name: CAT_Ret, dtype: float64

STD: step 4

In [27]:
sum_of_squares = sum(deviation_squares)

In [28]:
sum_of_squares

0.012159883712233545

STD: step 5

In [29]:
variance =  sum_of_squares / (5 - 1)

In [30]:
variance

0.003039970928058386

STD: step 6

In [31]:
stdv = np.sqrt(variance)

In [32]:
stdv

0.055135931370190766

# **Create_Stat_Board**

In [33]:
def stat_board(symbols):
  columns = [
      'annualized_mean_return',
      'annualized_std',
      'sharp_ratio'
  ]
  board = pd.DataFrame(index=columns)
  for symbol in symbols:
    board[symbol] = [
        df[f'{symbol}_Ret'].mean() * 12,
        df[f'{symbol}_Ret'].std() * np.sqrt(12),
        (df[f'{symbol}_Ret'].mean() * 12) / (df[f'{symbol}_Ret'].std() * np.sqrt(12))
        ]


  return board


In [34]:
stat_board(symbols)

,GOOG,CAT
annualized_mean_return,0.220,0.155
annualized_std,0.319,0.337
sharp_ratio,0.690,0.460


In [39]:
board = stat_board(symbols)

# **Create_Portfolio_Board**

In [40]:
w = [0.5, 0.5]
mean = board[symbols[0]].annualized_mean_return * w[0] + board[symbols[1]].annualized_mean_return * w[1]
cov = df[f'{symbols[0]}_Ret'].cov(df[f'{symbols[1]}_Ret'])
std = np.sqrt(
    w[0]**2 *board[symbols[0]].annualized_std**2 +
    w[1]**2 *board[symbols[1]].annualized_std**2 +
    2 * w[0] * w[1] * cov
)
sr = mean / std 
sr

def portfolio_board(symbols):
  print('Portfolio Weights: \n', w)
  columns = [
      'portfolio_annualized_mean',
      'portfolio_annualized_std',
      'portfolio_sharp_ratio'
  ]
  portfolio_board = pd.DataFrame(index=columns) 
  portfolio_board['Portfolio Stat'] = [
      mean,
      std,
      sr
  ]
  print(portfolio_board)

In [41]:
std

0.23363149281472045

In [42]:
portfolio_board(symbols)

Portfolio Weights: 
 [0.5, 0.5]
                           Portfolio Stat
portfolio_annualized_mean           0.187
portfolio_annualized_std            0.234
portfolio_sharp_ratio               0.802
